<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notebooks/try_it_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
pip install jikanpy && pip install surprise

     |████████████████████████████████| 1.2MB 4.1MB/s 
     |████████████████████████████████| 153kB 19.7MB/s 
     |████████████████████████████████| 256kB 20.8MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=3d08797215393b599659cda182d9d123c123879e76318cc09e4b4efc2cfb9251
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [0]:
import pandas as pd

from jikanpy import Jikan
from surprise import accuracy
from surprise.dump import load

In [0]:
jikan = Jikan()

In [0]:
users = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/UserList.csv")
users = users[["username"]]
users = users.dropna()
users["lower"] = users["username"].apply(lambda x: x.lower())
users = users.set_index("lower")

In [0]:
users = dict(users.T)

In [0]:
def get_uid(username):
  username = username.lower()
  if username in users:
    return users[username.lower()][0]
  else:
    return username

In [123]:
anime = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/AnimeList.csv", index_col="anime_id")[["title",	"image_url"]]
anime.reset_index()[["anime_id"]]

,anime_id
0,11013
1,2104
2,5262
3,721
4,12365
...,...
14473,26089
14474,21525
14475,37897
14476,34193


In [0]:
surprised = load("/content/drive/My Drive/svd.pickle")

In [0]:
algo = surprised[1]
predictions = surprised[0]

In [8]:
accuracy.rmse(predictions)

RMSE: 1.1859


1.1859182533000143

In [0]:
orionite_completed = pd.DataFrame(jikan.user(username=user, request='animelist', argument='completed', page=1)["anime"])
orionite_completed = orionite_completed.append(pd.DataFrame(jikan.user(username=user, request='animelist', argument='completed', page=2)["anime"]))

In [245]:
orionite_completed[orionite_completed["title"].str.contains("Tengen")]

,mal_id,title,video_url,url,image_url,type,watching_status,score,watched_episodes,total_episodes,airing_status,season_name,season_year,has_episode_video,has_promo_video,has_video,is_rewatching,tags,rating,start_date,end_date,watch_start_date,watch_end_date,days,storage,priority,added_to_list,studios,licensors
107,2001,Tengen Toppa Gurren Lagann,https://myanimelist.net/anime/2001/Tengen_Topp...,https://myanimelist.net/anime/2001/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/4/512...,TV,2,10,27,27,2,None,None,False,True,True,False,None,PG-13,2007-01-04T00:00:00+00:00,2009-06-09T00:00:00+00:00,2009-01-06T00:00:00+00:00,2009-06-09T00:00:00+00:00,98.0,None,Low,False,[],[]
108,4107,Tengen Toppa Gurren Lagann Movie 1: Gurren-hen,https://myanimelist.net/anime/4107/Tengen_Topp...,https://myanimelist.net/anime/4107/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/10/11...,Movie,2,9,1,1,2,None,None,False,True,True,False,None,PG-13,2008-06-09T00:00:00+00:00,2008-06-09T00:00:00+00:00,2010-11-04T00:00:00+00:00,2010-11-04T00:00:00+00:00,1.0,None,Low,False,[],[]
109,4565,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,https://myanimelist.net/anime/4565/Tengen_Topp...,https://myanimelist.net/anime/4565/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/12/19...,Movie,2,10,1,1,2,None,None,False,False,False,False,None,R+,2010-10-04T00:00:00+00:00,2010-10-04T00:00:00+00:00,2012-06-01T00:00:00+00:00,2012-06-01T00:00:00+00:00,1.0,None,Low,False,[],[]
110,5947,Tengen Toppa Gurren Lagann Movie Zenyasai: Vir...,https://myanimelist.net/anime/5947/Tengen_Topp...,https://myanimelist.net/anime/5947/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/13/22...,OVA,2,8,1,1,2,None,None,False,False,False,False,None,PG-13,2008-12-01T00:00:00+00:00,2008-12-01T00:00:00+00:00,2010-09-01T00:00:00+00:00,2010-09-01T00:00:00+00:00,1.0,None,Low,False,[],[]
111,6548,Tengen Toppa Gurren Lagann: Kirameki★Yoko Box ...,https://myanimelist.net/anime/6548/Tengen_Topp...,https://myanimelist.net/anime/6548/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/9/249...,Music,2,8,1,1,2,None,None,False,False,False,False,None,PG-13,2011-03-05T00:00:00+00:00,2011-03-05T00:00:00+00:00,None,None,NaN,None,Low,False,[],[]
112,3352,Tengen Toppa Gurren Lagann: Mitee Mono wa Mite...,https://myanimelist.net/anime/3352/Tengen_Topp...,https://myanimelist.net/anime/3352/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/9/196...,Special,2,8,0,1,2,None,None,False,False,False,False,None,PG-13,2010-02-02T00:00:00+00:00,2010-02-02T00:00:00+00:00,2007-04-12T00:00:00+00:00,2007-04-12T00:00:00+00:00,1.0,None,Low,False,[],[]
113,4705,Tengen Toppa Gurren Lagann: Parallel Works,https://myanimelist.net/anime/4705/Tengen_Topp...,https://myanimelist.net/anime/4705/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/5/109...,Music,2,9,8,8,2,None,None,False,False,False,False,None,PG-13,2009-03-06T00:00:00+00:00,2009-02-09T00:00:00+00:00,2009-03-06T00:00:00+00:00,2009-02-09T00:00:00+00:00,92.0,None,Low,False,[],[]
114,8348,Tengen Toppa Gurren Lagann: Parallel Works 2,https://myanimelist.net/anime/8348/Tengen_Topp...,https://myanimelist.net/anime/8348/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/13/23...,Music,2,8,7,7,2,None,None,False,False,False,False,None,PG-13,2012-02-05T00:00:00+00:00,2012-02-05T00:00:00+00:00,2010-04-06T00:00:00+00:00,2010-04-06T00:00:00+00:00,1.0,None,Low,False,[],[]


In [250]:
orionite_completed_lst = list(orionite_completed["mal_id"])

2001 in orionite_completed_lst

True

In [251]:
user = get_uid("orionite")

user_orionite = anime.reset_index()[["anime_id"]]

user_orionite["est"] = user_orionite["anime_id"].apply(lambda item: algo.predict(user, item)[3] if item not in orionite_completed_lst else 0)

user_orionite.sort_values("est", ascending=False).head(10)["anime_id"].apply(lambda x: anime.loc[x])

,title,image_url
7483,Redline,https://myanimelist.cdn-dena.com/images/anime/...
4687,Detroit Metal City,https://myanimelist.cdn-dena.com/images/anime/...
3677,Prison School,https://myanimelist.cdn-dena.com/images/anime/...
10598,Daicon Opening Animations,https://myanimelist.cdn-dena.com/images/anime/...
10810,JoJo no Kimyou na Bouken (TV),https://myanimelist.cdn-dena.com/images/anime/...
13232,Nihon Animator Mihonichi,https://myanimelist.cdn-dena.com/images/anime/...
9889,Great Teacher Onizuka,https://myanimelist.cdn-dena.com/images/anime/...
6687,Mahou Shoujo Lyrical Nanoha: The Movie 2nd A&#...,https://myanimelist.cdn-dena.com/images/anime/...
5005,Hellsing Ultimate,https://myanimelist.cdn-dena.com/images/anime/...
9880,Uchuu Senkan Yamato 2199,https://myanimelist.cdn-dena.com/images/anime/...


In [178]:
user_orionite

,anime_id,est
0,11013,7.910471
1,2104,9.330880
2,5262,7.880556
3,721,7.892112
4,12365,8.488308
...,...,...
14473,26089,7.376596
14474,21525,7.705882
14475,37897,7.706382
14476,34193,7.706382


In [224]:
# user factors (n_users, n_factors)
algo.pu[0]

array([-0.10964764,  0.0644132 ,  0.07801158,  0.18785343,  0.13044843,
        0.0816009 , -0.04354965, -0.13049902,  0.29556624, -0.18943162,
        0.15343936,  0.11352456, -0.12047465, -0.01193312, -0.00639427,
       -0.24816709,  0.04486322, -0.03175322, -0.05919502,  0.26406707,
       -0.05431184, -0.10821562, -0.22316178, -0.01923154, -0.07012862,
        0.32352119, -0.18648226,  0.03759529,  0.08528246, -0.07084615,
       -0.05253177, -0.03210823,  0.1763752 , -0.0618852 ,  0.228194  ,
        0.19996405, -0.11953084, -0.34856987, -0.15103973,  0.16548378,
        0.11159979,  0.06441523,  0.01045672, -0.07757395,  0.03409468,
        0.06488721,  0.00360526, -0.17637929,  0.08903039,  0.17047077,
        0.06997993, -0.55025234,  0.0093419 ,  0.08678235,  0.32406392,
       -0.1440272 , -0.04432669, -0.05377477,  0.15502467, -0.02025733,
        0.05649897,  0.22920751, -0.11284086,  0.13677604,  0.11792899,
       -0.35910372, -0.00715197, -0.39200042,  0.17792549, -0.09

In [225]:
# item factors (n_items, n_factors)
algo.qi[0]

array([ 0.42709911, -0.28117853, -0.06381595, -0.05734433, -0.07931106,
        0.42733028, -0.34496157, -0.24146143,  0.26566008,  0.10157331,
       -0.69469139, -0.40108364,  0.16823679,  0.06035036,  0.13880951,
       -0.17149166,  0.39679193,  0.23724567, -0.02829183, -0.06513415,
       -0.09182997,  0.32917318, -0.29527852,  0.11351543,  0.41465146,
        0.32349009,  0.42206472,  0.1376204 ,  0.32755613,  0.31566931,
       -0.00895321,  0.01206662,  0.23039966,  0.55485186, -0.2266337 ,
       -0.3160523 ,  0.15437517, -0.08905405,  0.41339608,  0.24516277,
        0.25700383,  0.46243531, -0.1665267 ,  0.18155341,  0.34524059,
       -0.08826208, -0.08534759,  0.10599066, -0.16018633,  0.4427061 ,
        0.31034535, -0.3084262 ,  0.20602024,  0.0273474 , -0.07117152,
        0.38642278, -0.12666264,  0.02532184,  0.20931989,  0.16916658,
       -0.52505023, -0.12093556, -0.14442767,  0.20892785,  0.13101318,
       -0.27524866,  0.06997981, -0.12183319, -0.13099155, -0.16

In [226]:
# user biases (n_users)
algo.bu[0]

0.25164323312979164

In [227]:
# item biases (n_items)
algo.bi[0]

-0.056219577209310805

{}